### CS4102 - Geometric Foundations of Data Analysis I
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Week 4

## 0. Questions

* How to compute the $r^2$ for the skin care example?
* How to compute simultaneous interval estimates?
* How to create an archive of 3 files?

## Archives

* zip
* tar

## Coefficient of Determination: Spare Parts Example

* The data are in a CSV file `production.csv`.

In [ ]:
%cat production.csv

* To actually read the data into the python sessions, we can use the `DictReader` command in the `csv` library,
after importing that library

In [ ]:
import csv
with open('production.csv') as csvfile:
    rows = list(csv.DictReader(csvfile))

* Now each item in the list `rows` is a python dictionary with fields `i`, `x`, `y`, corresponding to the headers of the CSV file.  
* As python starts counting at $0$, the row for $i = 4$ is the list item at position $3$.

In [ ]:
rows[3]

* Python's *list comprehension* can be used to store all the `x`-values in a list.  
* Note however, that these values are strings and not integers: `'80'` rather than `80`.

In [ ]:
[row['x'] for row in rows]

* We use the `numpy` package to convert such a list of strings into a list of *floating point values*.
* The *keyword argument* `dtype` of the `array` constructor can be used to specify the *data type* that `numpy` should use throughout the array it builds.
* $Y$ s simply the list of all $y$-values in the dictionary `rows`.

In [ ]:
import numpy as np
Y = np.array([row['y'] for row in rows], dtype=float)
Y

* $X$ should be a matrix, or an $n \times 2$-array, consisting of rows of the form $[1,x]$, one for each
$x$-value in the table.

In [ ]:
X = np.array([[1, row['x']] for row in rows], dtype=float)
X

* Next, we compute the column vector $B$ consisting of the coefficients $b_0$ and $b_1$.  According to the notes,
the formula is
$$
  B = (X^t X)^{-1} X^t Y
$$
* We first prepare the matrices $X^t X$ and $X^t Y$, using `numpy`'s `@` operator for matrix multiplication
(as opposed to `*` which does something different) and the `T` method for the matrix transpose.

In [ ]:
XtX = X.T @ X  # T for transpose, @ for matrix multiplication
XtY = X.T @ Y
print(XtX)

In [ ]:
print(XtY)

* Finally, we can use `np.linalg.inv` to invert the matrix `XtX`, and compute $B$ as the matrix product of that inverse and `XtY`.

In [ ]:
B = np.linalg.inv(XtX) @ XtY
print(B)

* Now $B = (b_0, b_1)^t$ represents the least squares fit $y = b_0 + b_1 x$.
* We can use it to compute a $y$-value from any given $x$-value.
* Note that, with the row vector $(1, x)$, this value is a *matrix product*:
$b_0 + b_1 x = (1, x) (b_0, b_1)^t$.

In [ ]:
[1, 40] @ B

* In particular, we can use $B$ to compute the values $\hat{y}_i = b_0 + b_1 x_i$, e.g., $\hat{y}_4 = b_0 + b_1 x_4$, and compare these to the original values $y_i$.

In [ ]:
print(X[4] @ B)
print(Y[4])

* In fact, we can compute the entire matrix $\hat{Y}$ as a single matrix product ...

In [ ]:
Yhat = X @ B
Yhat

* ... and then compute the matrix $E$ with entries $e_i = y_i - \hat{y}_i$ as a difference of two matrices.

In [ ]:
E = Y - Yhat
E

* The *mean* of the $y$-values is the average $\bar{y} = \frac1n \sum y_i$.  That is the sum of all $y$-values, divided by their number.

In [ ]:
ybar = sum(Y)/len(Y)
ybar

* In the notes, the quantities used for the **coefficient of determination** were defined as follows.
* $\mathrm{SSTO} = \sum (y_i - \bar{y})^2$
* $\mathrm{SSE} = \sum (y_i - \hat{y}_i)^2$
* $\mathrm{SSR} = \sum (\hat{y}_i - \bar{y})^2$

* With numpy, we can subtract a number (like `ybar`) from (each entry of) an array by simply computing the difference.  
* This gives a column vector $Y - \bar{y}$ of values $y_i - \bar{y}$.

In [ ]:
Y - ybar

* So, SSTO is the sum of the squares of the above numbers.
* In numpy, the square of an array is the array consisting of the squares of the original entries. (Read this again!  This is different from the square of a matrix in the matrix multiplication sense!)
* And recall that: exponentiation in Python is done with the `**` operator.

In [ ]:
(Y - ybar)**2

* So, SSTO is the sum of the square of the entries of the column vector $Y - \bar{y}$.

In [ ]:
SSTO = sum((Y - ybar)**2)
SSTO

* And SSE is the sum of the squares of the entries of the column vector $Y - \hat{Y}$, which we chose to call $E$ earlier.

In [ ]:
SSE = sum(E**2)
SSE

* And SSR is the sum of the squares of the column vector $\hat{Y} - \bar{y}$.

In [ ]:
SSR = sum((Yhat - ybar)**2)
SSR

* And finally, $r^2$ is the quotient of SSR and SSTO.

In [ ]:
r2 = SSR/SSTO
r2

* An $r^2$ value close to $1$ means that the variance in $y$ is quite well explained by regression (in $x$-direction).

##  Hypothesis Testing: Skin Care Example

* We can apply almost the same  procedure as above, to find the coefficient of determination for the data in the file `cream.csv`.
* There is no need to import the packages (`csv` and `numpy`) again.

In [ ]:
with open('cream.csv') as csvfile:
    rows = list(csv.DictReader(csvfile))

* The main difference is that the data file now has one more column of $x$-values.

In [ ]:
rows[0]

* Still, we can build numpy arrays `X` and `Y`, representing the matrices $X$ and $Y$, in a similar fashion as before.

In [ ]:
X = np.array([[1, row['xone'], row['xtwo']] for row in rows], dtype=float)
Y = np.array([row['y'] for row in rows], dtype=float)

* The matrix formula for computing the coefficients $B = (b_0, b_1, b_2)^t$ of the least squares fit $y = b_0 + b_1 x_1 + b_2 x_2$ is still the same:
$$
  B = (X^t X)^{-1} X^t Y
$$
* And so is the sequence of steps used to compute it.

In [ ]:
XtX = X.T @ X  # T for transpose, @ for matrix multiplication
XtY = X.T @ Y
B = np.linalg.inv(XtX) @ XtY
print(B)

* From this, we compute the quantities SSE, SSR and SSTO as before ...

In [ ]:
Yhat = X @ B
E = Y - Yhat
ybar = np.mean(Y)
SSTO = sum((Y - ybar)**2)
SSE = sum(E**2)
SSR = sum((Yhat - ybar)**2)
print("SSR =", SSR, ", SSE =", SSE, ", SSTO =", SSTO)

* Finally $r^2 = \mathrm{SSR}/\mathrm{SSTO}$

In [ ]:
r2 = SSR/SSTO
r2

* Next, the F-test requires slightly modified quantities:
$$
\mathrm{MSR} = \frac{\mathrm{SSR}}{p-1}, \qquad
\mathrm{MSE} = \frac{\mathrm{SSE}}{n-p}, \qquad
F^* = \frac{\mathrm{MSR}}{\mathrm{MSE}}
$$
* Here, $p = 3$ and $n = 15$.
* Let's compute $F^*$!

In [ ]:
n = len(X)
p = len(X[0])
print("n =", n, ", p =", p)
MSR = SSR/(p-1)
MSE = SSE/(n-p)
Fstar = MSR/MSE
Fstar

* Then, assuming that the errors $\epsilon_i$ are independent $N(0, \sigma^2)$, we choose a confidence level $\alpha = 0.05$ and find the value of the $F$-distribution with $p-1$ and $n-p$ degrees of freedom.
* This value can be found in a table, online or off-line, or with the help of the `scipy.stats` package

In [ ]:
from scipy.stats import f
alpha = 0.05
f.pdf(1 - alpha, p-1, n-p)

* As this value is clearly smaller than $F^*$, we can reject the null hypothesis $\mathcal{C}_0$ at level $\alpha$.

* In order to quickly check whether the $\epsilon_i$ are independent and normally distributed, we can plot them (i) against the actual error $\hat{y}_i$, (ii) against the input data $x_{i1}$, (iii) against the input data $x_{i2}$.
* The $x_{i1}$ reside in column $1$ of the array `X`, from where we can extract them as `X[:,1]`, using a *slice* (`:` for all) in the first dimension, and an index (`1` for column $1$) in the second dimension.

In [ ]:
X[:,1]

* For plotting, we use the `matplotlib.pyplot` package under its nickname `plt`.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Yhat = X @ B
E = Y - Yhat
plt.plot(Yhat, E, 'b.')

In [ ]:
plt.plot(X[:,1], E, 'go')

In [ ]:
plt.plot(X[:,2], E, 'r+')

* The **estimated covariance matrix** for the least squares model 
$$
y_i = \beta_0 + \beta_1 x_{i,1} + \dots + \beta_{p-1} x_{i,p-1} + \epsilon_i
$$
is $S^2(B) = \mathrm{MSE} (X^t X)^{-1}$.

In [ ]:
S2B = MSE * np.linalg.inv(XtX)
S2B

* Theory says that if $q$ of the $\beta_k$ are jointly estimated, the confidence intervals
with coefficient $1 - \alpha$ are
$$
b_k - T \cdot s(b_k) \leq \beta_k \leq b_k + T \cdot s(b_k),
$$
where $T = t(1 - \frac{\alpha}{2q}, n - p)$.
* The value of the $T$-distribution can be found in a table, online or off-line, or again with the help of the `scipy.stats` package.

In [ ]:
from scipy.stats import t
q = 2
T = t.pdf(1 - alpha/2/q, n - p)
T

* So, when estimating $\beta_1$ and $\beta_2$ jointly, after extracting the values $s(b_k)$ as square roots of the diagonal values of the array `S2B`, we can find the *lower bounds* of the confidence intervals for the $\beta_k$ as follows.

In [ ]:
SB = np.diagonal(S2B)**0.5
B - T * SB

* And the *upper bound*:

In [ ]:
B + T * SB

## Exercises

* Find and study the **documentation** for those elements of python in this notebook which are new to you.

* Find $r^2$, $F^*$, and the $\beta_k$ confidence intervals for the least squares straight line fit in the parabola example.